<a href="https://colab.research.google.com/github/izzul112/ML_MLO-Submission_1/blob/main/mlop_submission1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Tahap Persiapan**

Di tahap ini kita akan menginsatall semua library yang di butuhkan serta membuat virtual environment.

1. Install condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

2. Cek versinya jika perlu

In [ ]:
!conda --version

3. Buat conda virtual environment, kita berinama mlops-tfx-sub-1

In [ ]:
!conda create --name mlops-tfx-sub-1 python==3.9.15

4. Aktifkan conda virtual environment-nya

In [ ]:
!conda activate mlops-tfx-sub-1

In [2]:
!git clone https://github.com/izzul112/ML_MLO-Submission_1.git

Cloning into 'ML_MLO-Submission_1'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 22 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), 6.76 KiB | 1.35 MiB/s, done.
